### Installation

In terminal:

```  
conda create -n starfish python=3.7  
conda activate starfish  
pip install starfish[napari]  
```

In [ ]:
from starfish import data

import tifffile as tif
from glob import glob
import os
import pandas as pd

import csv
import os
import numpy as np
import shutil
import skimage.io
import tempfile
import time
import sys
import numpy as np
import copy
from scipy import spatial

from slicedimage import ImageFormat
from starfish.experiment.builder import format_structured_dataset

from starfish import Experiment
from starfish.types import Axes

# Decode spots with SimpleLookupDecoder
from starfish.spots import DecodeSpots
from starfish.spots import FindSpots


In [ ]:
## analysis type: '' (for the original analysis) OR 'adj_spots'
analysis = '_adj_spots'
analysis = ''

### Set paths

In [ ]:
# For comparison of similar number of points as RSFISH found for each embryo, csv of data from RSFISH
# Assuming that more points take longer to analyse

#df_RSFISH_results_path = 'PATH_TO/RSFISH_embryos_npoints_and_times.csv'

In [ ]:
#results_embryos_df_path = 'PATH/TO/DESIRED/LOCATION/OF/EMBRYOS/RESULTS.csv'

### Set original images dir 

In [ ]:
path = '/YOUR/PATH/HERE'

In [ ]:
org_ims_dir = os.path.join(path, 'ORG_IMS')
org_ims_dir = os.path.join(path, f'simulations{analysis}')

In [ ]:
org_ims_paths = glob(os.path.join(org_ims_dir,'*','*.tif'))

In [ ]:
org_ims_paths[1:4]

In [ ]:
# Those images are used for accuracy analysis
org_simul_ims_paths = [p for p in org_ims_paths 
                       if "embryos_FISH" not in p
                      and "3000spots" not in p
                      and os.path.exists(f'{p[:-4]}.loc')]

## Those images are used for execution time analysis:
org_embryo_ims_paths = [p for p in org_ims_paths if "embryos_FISH" in p]

### Create analysis dirs

In [ ]:
sf_dir = os.path.join(path, 'starfish')
os.makedirs(sf_dir, exist_ok=True)

In [ ]:
sf_prep_ims_dir = os.path.join(sf_dir, f'prep_spacext_ims{analysis}')
os.makedirs(sf_prep_ims_dir, exist_ok=True)

In [ ]:
spacext_ims_dir = os.path.join(sf_dir, f'spacext_ims{analysis}')
os.makedirs(spacext_ims_dir, exist_ok=True)

In [ ]:
results_path_simul = os.path.join(sf_dir, f'results{analysis}')
os.makedirs(results_path_simul, exist_ok=True)

## Convert Image Data to SpaceTX Format 

In [ ]:
for im_path in org_ims_paths: 

    im = tif.imread(im_path)
    
    # columns: r, ch, zplane
    fovs = [
        [
            (0, 0, i) for i in range(im.shape[0])
        ],
    ]

    im_dir = os.path.basename(os.path.dirname(im_path))
    
    # Time analysis done on the embryo dir:
    if im_dir=='embryos_FISH':
        zc_max = 358.0
    # accuracy analysis done on simulated data
    elif 'Sigxy' in im_dir:
        str_xy = im_dir.split('Sigxy ')[1].split(' SigZ')[0]
        str_xy = 1.0+float('0.'+str_xy.split('pt')[1]) if 'pt' in str_xy else int(str_xy)
        str_z = int(im_dir.split('SigZ ')[1])
        
        zc_max = 256.0*(str_z/str_xy)
    else:
        str_xy = float(im_path.split('sxy')[1].split('_sz')[0])
        str_z =  float(im_path.split('_sz')[1].split('/')[0])
        
        zc_max = 256.0*(str_z/str_xy)
    
    coordinates_of_fovs = [
        {
            'xc_min': 0.0,
            'xc_max': 256.0,
            'yc_min': 0.0,
            'yc_max': 256.0,
            'zc_min': 0.0,
            'zc_max': zc_max,
        },
    ]

    # create example image tiles that adhere to the structured data schema
    primary_dir = os.path.join(sf_prep_ims_dir, os.path.basename(os.path.dirname(im_path)), os.path.basename(im_path)[:-4], "primary_dir")
    os.makedirs(primary_dir, exist_ok=True)

    for fov_id, fov in enumerate(fovs):
        for round_label, ch_label, zplane_label in fov:
            primary_path = os.path.join(
                primary_dir, f"primary-f{fov_id}-r{round_label}-c{ch_label}-z{zplane_label}.tiff")
            skimage.io.imsave(primary_path, im[zplane_label])

    # write coordinates file for primary:
    with open(os.path.join(primary_dir, "coordinates.csv"), "w") as fh:
        csv_writer = csv.DictWriter(
            fh,
            [
                'fov', 'round', 'ch', 'zplane',
                'xc_min', 'yc_min', 'zc_min', 'xc_max', 'yc_max', 'zc_max',
            ]
        )
        csv_writer.writeheader()
        for fov_id, (fov_info, coordinate_of_fov) in enumerate(zip(fovs, coordinates_of_fovs)):
            for round_label, ch_label, zplane_label in fov:
                tile_coordinates = coordinate_of_fov.copy()
                tile_coordinates.update({
                    'fov': fov_id,
                    'round': round_label,
                    'ch': ch_label,
                    'zplane': zplane_label,
                })
                csv_writer.writerow(tile_coordinates)
                
                
                
    primary_out = os.path.join(spacext_ims_dir, os.path.basename(os.path.dirname(im_path)), os.path.basename(im_path)[:-4], "primary")
    os.makedirs(primary_out, exist_ok=True)

    format_structured_dataset(
        primary_dir,
        os.path.join(primary_dir, "coordinates.csv"),
        primary_out,
        ImageFormat.TIFF,
    )

## Finding Spots 

#### Define spot finding function

We use blob_detector, but stardist has many options


In [ ]:
def find_spots(imgs, min_sigma, max_sigma, num_sigma, threshold):

    p = FindSpots.BlobDetector(
        min_sigma=min_sigma,
        max_sigma=max_sigma,
        num_sigma=num_sigma,
        threshold=threshold,
        measurement_type='mean', 
        exclude_border=False
    )
    
    intensities = p.run(image_stack=imgs, n_processes=40)
    return intensities

### Simulated Data 

### Quick Grid Search
on the parameters (sigma and threshold) to try to optimize blob detection on the simulated images

In [ ]:
## Some of those more specific values were chosen after the first run.
sigs = [2,4,5]
thrs = [0.001, 0.002, 0.0001, 0.000145, 0.000033, 0.0000404, 0.0000412, 0.000042, 0.0000428, 0.000043, 0.0000439, 0.000044, 0.000045, 0.000046, 
        0.000047, 0.000048, 0.0000483, 0.000049, 0.00005, 0.000065125, 0.000074, 0.000075, 0.000076, 0.000077, 
        0.000078, 0.000079, 0.00008, 0.0000805, 0.00009, 0.000095]

for i,im_path in enumerate(org_simul_ims_paths):
    
    if i>230:

        im_dir = os.path.basename(os.path.dirname(im_path))
        im_name = os.path.basename(im_path[:-4])

        # load image:
        experiment = Experiment.from_json(os.path.join(
        spacext_ims_dir, im_dir, im_name, 'primary', 'experiment.json'))

        imgs = experiment.fov().get_image('primary')

        # Find spots:
        print(im_path, i)

        for sig in sigs:

            for thr in thrs:

                start = time.time()

                spotss = find_spots(imgs, 0, sig, sig, thr)

                if analysis=="":
                    gt_n_spots = int(im_name.split('Poiss_')[1].split('spots')[0])
                else:
                    gt_n_spots = 30

                if abs( gt_n_spots - spotss.count_total_spots() ) == 0:
                    flag =True

                diff_nspots = abs(gt_n_spots - spotss.count_total_spots())

                #print(f'gt_nspots - nspots = {diff_nspots}')

                if diff_nspots < 25:

                    # Decode:
                    decoder = DecodeSpots.SimpleLookupDecoder(codebook=experiment.codebook)
                    decoded_intensities = decoder.run(spots=spotss)

                    exe_time = time.time() - start
                    exe_time = f'{exe_time:.3g}'

                    filename = f'{im_dir}__{im_name}__sig{sig}_thr{thr}__exetime{exe_time}.csv'

                    decoded_intensities.to_dataframe("results")[["x","y","z"]].reset_index(drop=True).to_csv(
                    os.path.join(results_path_simul,filename))
